In [ ]:
import glob
import os

from pymatreader import read_mat
import numpy as np


data_dir = '../data/train'

In [3]:
fold0_list = []
fold1_list = []
fold2_list = []

for subject in range(5):
    label_dict = {}
    for i, mat_data in enumerate(glob.glob(f'{data_dir}/subject{subject}/*')):
        data = read_mat(mat_data)
        data = data['data']
        if (subject == 0) or (subject == 3):
            if 'train1' in mat_data:
                fold0_list.append(data)
            elif 'train2' in mat_data:
                fold1_list.append(data)
            elif 'train3' in mat_data:
                fold2_list.append(data)
        elif (subject == 1) or (subject == 4):
            if 'train2' in mat_data:
                fold0_list.append(data)
            elif 'train3' in mat_data:
                fold1_list.append(data)
            elif 'train1' in mat_data:
                fold2_list.append(data)
        elif subject == 2:
            if 'train3' in mat_data:
                fold0_list.append(data)
            elif 'train1' in mat_data:
                fold1_list.append(data)
            elif 'train2' in mat_data:
                fold2_list.append(data)
            
for fold in range(3):
    dir_name = f'../../data/scaler3/'
    os.makedirs(f'{dir_name}', exist_ok=True)
    if fold == 0:
        data_list = fold1_list + fold2_list
    elif fold == 1:
        data_list = fold0_list + fold2_list
    elif fold == 2:
        data_list = fold0_list + fold1_list

    iqr_list, median_list = [], []
    for data in data_list:
        q1 = np.percentile(data, 25, axis=1)
        q3 = np.percentile(data, 75, axis=1)
        iqr = q3 - q1
        median = np.median(data, axis=1)
        iqr_list.append(iqr)
        median_list.append(median)

    iqr = np.mean(np.stack(iqr_list), axis=0)
    iqr = np.where(iqr==0, 1, iqr)
    median = np.mean(np.stack(median_list), axis=0)

    np.save(f'{dir_name}/iqr{fold}', iqr)
    np.save(f'{dir_name}/median{fold}', median)